In [1]:
import pandas as pd
import numpy as np
import geopandas as gp
import shapely
from planet4 import io, markings

In [2]:
from coverage_utils import CoverageCalculator, get_obsids_for_region
from my_io import get_tilecoords, get_metadata, get_region_names

In [3]:
olddata = pd.read_csv("n_obj_df.csv")

In [4]:
olddata

,Latitude,Ls,MY,Which,fraction,n_objects,obsid,pixel_area,region_name
0,-85.4074,214.785,29,both,0.115,933,ESP_012079_0945,2.551911e+07,Macclesfield
1,-85.4065,183.727,29,both,0.070,1971,ESP_011407_0945,2.592302e+06,Macclesfield
2,-85.4056,198.040,29,both,0.072,1525,ESP_011723_0945,8.455380e+06,Macclesfield
3,-85.4049,207.892,29,both,0.168,788,ESP_011934_0945,7.357966e+06,Macclesfield
4,-85.4067,235.282,29,both,0.111,1285,ESP_012501_0945,4.882627e+06,Macclesfield
5,-85.4117,224.964,29,both,0.087,350,ESP_012290_0945,1.274910e+07,Macclesfield
6,-85.4086,183.683,29,both,0.074,453,ESP_011406_0945,3.767516e+06,Macclesfield
7,-85.4074,214.785,29,fans,0.088,781,ESP_012079_0945,1.942013e+07,Macclesfield
8,-85.4065,183.727,29,fans,0.046,1568,ESP_011407_0945,1.689222e+06,Macclesfield
9,-85.4056,198.040,29,fans,0.047,939,ESP_011723_0945,5.512495e+06,Macclesfield


In [5]:
data = pd.read_csv("P4_catalog_v1.1_tile_coords_final.csv")

In [286]:
7865/3

2621.6666666666665

In [285]:
data.BodyFixedCoordinateX.value_counts(dropna=False)

NaN            7865
 219.705037       1
 77.142883        1
 221.775759       1
-65.686409        1
-54.445869        1
 222.498339       1
 476.633376       1
 468.072076       1
 477.416816       1
-155.231403       1
 218.922983       1
-106.578993       1
 10.630504        1
-327.961453       1
 14.381824        1
 220.281940       1
-155.079249       1
 222.612662       1
-372.455625       1
 467.510836       1
 896.267656       1
 221.387972       1
-109.896246       1
 238.464911       1
-54.085377        1
 221.025511       1
 196.595188       1
-181.098864       1
 109.834441       1
               ... 
 471.454036       1
-895.647612       1
-371.350736       1
 901.091782       1
 197.277169       1
-50.640603        1
 9.544266         1
 221.827307       1
-55.829695        1
 221.183422       1
 112.117200       1
-155.429163       1
-284.213363       1
 113.308110       1
-188.246681       1
 220.529718       1
-26.676126        1
 470.120394       1
-66.944225        1


In [7]:
#x1 = data[data['x_tile'] == 1]

" Through all the coordinates for all tiles for an obsid into a Shapely object and let shapely define the boundary. This would be good to learn, b/c that’s exactly what shapely was made for and it is a standard GIS task to learn."

In [8]:
data[data['obsid'] == 'ESP_012079_0945'].x_tile.max()

5

In [9]:
allobs = data.obsid.unique() #give me every obsid

In [10]:

xmiba = []
xmaba = []
ymiba = []
ymaba = []

for obs in allobs:
    temp = data[data['obsid'] == obs] #look only at our obsid in question
    #Get xmins, xmaxes for the tiles
    xmi = temp.x_tile.min()
    xma = temp.x_tile.max()
    ymi = temp.y_tile.min()
    yma = temp.y_tile.max()
    xmiba.append(xmi)
    xmaba.append(xma)
    ymiba.append(ymi)
    ymaba.append(yma)
    
xmiba = np.asarray(xmiba)
ymiba = np.asarray(ymiba)
xmaba = np.asarray(xmaba)    
ymaba = np.asarray(ymaba)

Running and analyzing the above shows that x_tile has a min of 1, max of 5 and the y_tile has a min of 1, max of 145 across all our obsids.

In [11]:
results = 1

In [272]:
#similar to above, but now to create a dataframe which will help us get the corner values
bucket = []
for obs in allobs:
    d = {}
    temp = data[data['obsid'] == obs]
    d["x_tile_min"] = temp.x_tile.min()
    d["x_tile_max"] = temp.x_tile.max()
    d["y_tile_min"] = temp.y_tile.min()
    d["y_tile_max"] = temp.y_tile.max()
    d["obsid"] = obs
    d["top_left"] = (temp[(temp['x_tile'] == temp.x_tile.min()) & (temp["y_tile"] == temp.y_tile.min())].BodyFixedCoordinateX.values, temp[(temp['x_tile'] == temp.x_tile.min()) & (temp["y_tile"] == temp.y_tile.min())].BodyFixedCoordinateY.values)
    d["top_right"] = (temp[(temp['x_tile'] == temp.x_tile.min()) & (temp["y_tile"] == temp.y_tile.max())].BodyFixedCoordinateX.values, temp[(temp['x_tile'] == temp.x_tile.min()) & (temp["y_tile"] == temp.y_tile.max())].BodyFixedCoordinateY.values )
    d['bottom_left'] = (temp[(temp['x_tile'] == temp.x_tile.max()) & (temp["y_tile"] == temp.y_tile.min())].BodyFixedCoordinateX.values, temp[(temp['x_tile'] == temp.x_tile.max()) & (temp["y_tile"] == temp.y_tile.min())].BodyFixedCoordinateY.values)
    d['bottom_right'] = (temp[(temp['x_tile'] == temp.x_tile.max()) & (temp["y_tile"] == temp.y_tile.max())].BodyFixedCoordinateX.values, temp[(temp['x_tile'] == temp.x_tile.max()) & (temp["y_tile"] == temp.y_tile.max())].BodyFixedCoordinateY.values )
    bucket.append(d)
results = pd.DataFrame(bucket)


In [273]:
results

,bottom_left,bottom_right,obsid,top_left,top_right,x_tile_max,x_tile_min,y_tile_max,y_tile_min
0,"([-67.69881540000002], [255.79257630000004])","([-62.388268200000006], [269.3496037])",ESP_012079_0945,"([-68.3806735], [256.0561472])","([-63.070566500000005], [269.61576610000003])",5,1,109,1
1,"([108.67046979999999], [457.5703882])","([121.8381508], [477.67216169999995])",ESP_011697_0980,"([108.3633138], [457.7678756])","([121.5308617], [477.87006569999994])",2,1,91,1
2,"([nan], [nan])","([nan], [nan])",ESP_021455_0935,"([nan], [nan])","([nan], [nan])",5,1,63,1
3,"([-319.50996000000004], [-323.8713523])","([-328.12714460000007], [-328.13045330000006])",ESP_012008_0975,"([-319.1787501], [-324.5326963])","([-327.7960465], [-328.7914651])",5,1,72,1
4,"([227.52294890000002], [406.7287602])","([239.92119440000002], [417.6651198])",ESP_012436_0980,"([227.279471], [406.9997462])","([239.6778595], [417.937003])",2,1,63,1
5,"([-111.6923165], [131.7801493])","([-103.0975151], [148.7454417])",ESP_020347_0930,"([-112.0162199], [131.9435613])","([-103.4213403], [148.90914730000003])",2,1,72,1
6,"([nan], [nan])","([nan], [nan])",ESP_020519_0935,"([nan], [nan])","([nan], [nan])",2,1,72,1
7,"([-276.0157651], [-10.9464875])","([-288.3257547], [-3.5948949])",ESP_011931_0945,"([-276.20325180000003], [-11.2623309])","([-288.5132829], [-3.9105293])",2,1,54,1
8,"([-157.25998380000001], [24.800988])","([-153.8289426], [41.2989999])",ESP_011960_0925,"([-157.6200124], [24.8762236])","([-154.18845430000002], [41.37439929999999])",2,1,63,1
9,"([226.6364917], [406.2519646])","([241.1111681], [418.4473792])",ESP_012291_0980,"([226.39947599999996], [406.5285268])","([240.874321], [418.7246521])",2,1,72,1


Throw out NAN values, filter by the obsid's for each region, add that region name as a column so as to sort better.  
I think the bottom left and bottom right x values being diff. is okay, just curvature and that the tiles are mostly off-center from what I was thinking.

In [14]:
results.x_tile_min[0]

1

In [15]:
temp[(temp['x_tile'] == temp.x_tile.max()) & (temp["y_tile"] == temp.y_tile.max())].BodyFixedCoordinateX.values

array([221.6780802])

In [16]:
corners_df = results

In [17]:
corners_df.to_csv("Corners_df.csv")

Use olddata dataframe to find the obsids and their associated region names, add region names to appropriate obsids in newres and actually drop nan values

In [22]:
olddata.head()

,Latitude,Ls,MY,Which,fraction,n_objects,obsid,pixel_area,region_name
0,-85.4074,214.785,29,both,0.115,933,ESP_012079_0945,2.551911e+07,Macclesfield
1,-85.4065,183.727,29,both,0.070,1971,ESP_011407_0945,2.592302e+06,Macclesfield
2,-85.4056,198.040,29,both,0.072,1525,ESP_011723_0945,8.455380e+06,Macclesfield
3,-85.4049,207.892,29,both,0.168,788,ESP_011934_0945,7.357966e+06,Macclesfield
4,-85.4067,235.282,29,both,0.111,1285,ESP_012501_0945,4.882627e+06,Macclesfield


In [23]:
olddata.set_index("region_name").head()

,Latitude,Ls,MY,Which,fraction,n_objects,obsid,pixel_area
region_name,,,,,,,,
Macclesfield,-85.4074,214.785,29,both,0.115,933,ESP_012079_0945,2.551911e+07
Macclesfield,-85.4065,183.727,29,both,0.070,1971,ESP_011407_0945,2.592302e+06
Macclesfield,-85.4056,198.040,29,both,0.072,1525,ESP_011723_0945,8.455380e+06
Macclesfield,-85.4049,207.892,29,both,0.168,788,ESP_011934_0945,7.357966e+06
Macclesfield,-85.4067,235.282,29,both,0.111,1285,ESP_012501_0945,4.882627e+06


In [24]:
allregions = olddata.region_name.unique() #give me every region name

In [61]:
bucket = []
for region in allregions:
    d = {}
    d['region_name'] = region
    d['obsids'] = olddata[olddata['region_name']==region].obsid.unique()
    bucket.append(d)
name_obs = pd.DataFrame(bucket)

In [26]:
name_obs

,obsids,region_name
0,"[ESP_012079_0945, ESP_011407_0945, ESP_011723_...",Macclesfield
1,"[ESP_011697_0980, ESP_011341_0980, ESP_011842_...",Starburst
2,"[ESP_012884_0935, ESP_012251_0935, ESP_012739_...",Manhattan_Classic
3,"[ESP_012008_0975, ESP_011296_0975]",Wellington
4,"[ESP_012436_0980, ESP_012291_0980, ESP_012647_...",Albany
5,"[ESP_011565_0930, ESP_012211_0930, ESP_011776_...",Bilbao
6,"[ESP_011931_0945, ESP_012063_0945, ESP_011350_...",Ithaca
7,"[ESP_011960_0925, ESP_012817_0925, ESP_012316_...",Portsmouth
8,"[ESP_012077_0930, ESP_012499_0930, ESP_011431_...",Pisaq
9,"[ESP_011711_0930, ESP_012990_0930, ESP_012634_...",Manhattan_Frontinella


Want to make it so every obsid in results gets checked against name_obs to find the region name, append the region name to the results dataframe in a new column titled region_name

In [53]:
name_obs[name_obs['region_name'] == 'Macclesfield'].obsids

0    [ESP_012079_0945, ESP_011407_0945, ESP_011723_...
Name: obsids, dtype: object

In [60]:
results[results['obsid']=='ESP_012079_0945']

,bottom_left,bottom_right,obsid,top_left,top_right,x_tile_max,x_tile_min,y_tile_max,y_tile_min
0,"([-67.69881540000002], [-67.69881540000002])","([-62.388268200000006], [269.3496037])",ESP_012079_0945,"([-68.3806735], [256.0561472])","([-63.070566500000005], [269.61576610000003])",5,1,109,1


In [72]:
results['region_name'] = np.nan #add nan values column so as to add real region names later.

In [274]:
#if its in name_obs.obsids, add a column to results with the region name of that one.
for region in allregions:
    x = name_obs[name_obs['region_name']== region].obsids  #collect every obsid in a region
    x = x.values.tolist()
    for vals in x[0]:
        results.loc[results.obsid == vals, 'region_name'] = region
#line above should replace region name with the proper region if the obsid matches any in x, but it doesnt work
results 

,bottom_left,bottom_right,obsid,top_left,top_right,x_tile_max,x_tile_min,y_tile_max,y_tile_min,region_name
0,"([-67.69881540000002], [255.79257630000004])","([-62.388268200000006], [269.3496037])",ESP_012079_0945,"([-68.3806735], [256.0561472])","([-63.070566500000005], [269.61576610000003])",5,1,109,1,Macclesfield
1,"([108.67046979999999], [457.5703882])","([121.8381508], [477.67216169999995])",ESP_011697_0980,"([108.3633138], [457.7678756])","([121.5308617], [477.87006569999994])",2,1,91,1,Starburst
2,"([nan], [nan])","([nan], [nan])",ESP_021455_0935,"([nan], [nan])","([nan], [nan])",5,1,63,1,Manhattan_Classic
3,"([-319.50996000000004], [-323.8713523])","([-328.12714460000007], [-328.13045330000006])",ESP_012008_0975,"([-319.1787501], [-324.5326963])","([-327.7960465], [-328.7914651])",5,1,72,1,Wellington
4,"([227.52294890000002], [406.7287602])","([239.92119440000002], [417.6651198])",ESP_012436_0980,"([227.279471], [406.9997462])","([239.6778595], [417.937003])",2,1,63,1,Albany
5,"([-111.6923165], [131.7801493])","([-103.0975151], [148.7454417])",ESP_020347_0930,"([-112.0162199], [131.9435613])","([-103.4213403], [148.90914730000003])",2,1,72,1,Bilbao
6,"([nan], [nan])","([nan], [nan])",ESP_020519_0935,"([nan], [nan])","([nan], [nan])",2,1,72,1,Manhattan_Classic
7,"([-276.0157651], [-10.9464875])","([-288.3257547], [-3.5948949])",ESP_011931_0945,"([-276.20325180000003], [-11.2623309])","([-288.5132829], [-3.9105293])",2,1,54,1,Ithaca
8,"([-157.25998380000001], [24.800988])","([-153.8289426], [41.2989999])",ESP_011960_0925,"([-157.6200124], [24.8762236])","([-154.18845430000002], [41.37439929999999])",2,1,63,1,Portsmouth
9,"([226.6364917], [406.2519646])","([241.1111681], [418.4473792])",ESP_012291_0980,"([226.39947599999996], [406.5285268])","([240.874321], [418.7246521])",2,1,72,1,Albany


In [282]:
bad = results.loc[6].bottom_left
print(bad)
print(type(bad))
bad2 = results.loc[6].bottom_right
print(bad2)
bad == bad2 #that's why
#why are the nan tuples not the same?! and how to fix?

(array([nan]), array([nan]))
<class 'tuple'>
(array([nan]), array([nan]))


False

In [265]:
indexNames = results[results['bottom_left'] == bad].index
indexNames

Int64Index([6], dtype='int64')

In [283]:
len(indexNames) #why doesnt this give all of them? because of bad !=bad2 above

1

In [258]:
results.drop(indexNames, inplace = True) #almost what I want
#need to figure out why indexNames doesnt find the other nan values
#once I do that the dataframe should be complete.

KeyError: '[2] not found in axis'

In [276]:
results

,bottom_left,bottom_right,obsid,top_left,top_right,x_tile_max,x_tile_min,y_tile_max,y_tile_min,region_name
0,"([-67.69881540000002], [255.79257630000004])","([-62.388268200000006], [269.3496037])",ESP_012079_0945,"([-68.3806735], [256.0561472])","([-63.070566500000005], [269.61576610000003])",5,1,109,1,Macclesfield
1,"([108.67046979999999], [457.5703882])","([121.8381508], [477.67216169999995])",ESP_011697_0980,"([108.3633138], [457.7678756])","([121.5308617], [477.87006569999994])",2,1,91,1,Starburst
2,"([nan], [nan])","([nan], [nan])",ESP_021455_0935,"([nan], [nan])","([nan], [nan])",5,1,63,1,Manhattan_Classic
3,"([-319.50996000000004], [-323.8713523])","([-328.12714460000007], [-328.13045330000006])",ESP_012008_0975,"([-319.1787501], [-324.5326963])","([-327.7960465], [-328.7914651])",5,1,72,1,Wellington
4,"([227.52294890000002], [406.7287602])","([239.92119440000002], [417.6651198])",ESP_012436_0980,"([227.279471], [406.9997462])","([239.6778595], [417.937003])",2,1,63,1,Albany
5,"([-111.6923165], [131.7801493])","([-103.0975151], [148.7454417])",ESP_020347_0930,"([-112.0162199], [131.9435613])","([-103.4213403], [148.90914730000003])",2,1,72,1,Bilbao
6,"([nan], [nan])","([nan], [nan])",ESP_020519_0935,"([nan], [nan])","([nan], [nan])",2,1,72,1,Manhattan_Classic
7,"([-276.0157651], [-10.9464875])","([-288.3257547], [-3.5948949])",ESP_011931_0945,"([-276.20325180000003], [-11.2623309])","([-288.5132829], [-3.9105293])",2,1,54,1,Ithaca
8,"([-157.25998380000001], [24.800988])","([-153.8289426], [41.2989999])",ESP_011960_0925,"([-157.6200124], [24.8762236])","([-154.18845430000002], [41.37439929999999])",2,1,63,1,Portsmouth
9,"([226.6364917], [406.2519646])","([241.1111681], [418.4473792])",ESP_012291_0980,"([226.39947599999996], [406.5285268])","([240.874321], [418.7246521])",2,1,72,1,Albany


the NAN NAN tuples are unique(Why??) so the above method won't work, come back to after consulting with michael

In [277]:
print(results.loc[0].bottom_left)
print(results.loc[0].bottom_left[0])
print(results.loc[0].bottom_left[1])


(array([-67.6988154]), array([255.7925763]))
[-67.6988154]
[255.7925763]
